# Population Clusterization

*Author*: Marcos Bressan

The dataset used in this project was collected by the United States Census, and it provides information about characteristics of the demography of several counties in United States.

OBS: Previously, I was training my model on census data available on https://www.census.gov/data.html. 
A few months later, I discovered an example for the same project on the [SameMaker blog](https://aws.amazon.com/blogs/machine-learning/analyze-us-census-data-for-population-segmentation-using-amazon-sagemaker/).
It uses a public AWS's bucket to get the data from, which speeds up the download time and avoids more code for data pre-processing, since I'll be using Amazon AWS to train and deploy my model.

## Objectives
- Explore dataset
- Pre-process/treat data
- Apply Principal Components Analysis (PCA)
- Feature Engineering / Dataset Transformation
- Clusterization of transformed data
- Visualization and interpretation of results

In [3]:
import os
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline 
# from sagemaker
import boto3
import sagemaker

## Get data from S3 bucket

As mentioned on [Sagemaker blog](https://aws.amazon.com/blogs/machine-learning/analyze-us-census-data-for-population-segmentation-using-amazon-sagemaker/), the data is availabke in `aws-ml-blog-sagemaker-census-segmentation` S3 bucket, easely downloadable using the `boto3` module.

The desired file name is `Census_Data_for_SageMaker.csv`.

In [16]:
client = boto3.client('s3')

dataset_bucket = 'aws-ml-blog-sagemaker-census-segmentation'
dataset_filename = 'Census_Data_for_SageMaker.csv'

dataset_object = client.get_object(Bucket=dataset_bucket, Key=dataset_filename)

In [17]:
# Exploring dataset object
dataset_object

{'ResponseMetadata': {'RequestId': '9P9SCN5HDJ3Q6V1J',
  'HostId': '8UqCBwwb6B6pIDDCCKcwDZzGNLjw0KSyqrLsiQNRBwMN1lW1z+AAOmTYPpW6HPY4FoMmPMhdiM0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '8UqCBwwb6B6pIDDCCKcwDZzGNLjw0KSyqrLsiQNRBwMN1lW1z+AAOmTYPpW6HPY4FoMmPMhdiM0=',
   'x-amz-request-id': '9P9SCN5HDJ3Q6V1J',
   'date': 'Tue, 17 Nov 2020 20:48:15 GMT',
   'last-modified': 'Wed, 12 Sep 2018 15:13:37 GMT',
   'etag': '"066d37f43f7762f1eb409b1660fe9763"',
   'accept-ranges': 'bytes',
   'content-type': 'text/csv',
   'content-length': '613237',
   'server': 'AmazonS3'},
  'RetryAttempts': 1},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2018, 9, 12, 15, 13, 37, tzinfo=tzutc()),
 'ContentLength': 613237,
 'ETag': '"066d37f43f7762f1eb409b1660fe9763"',
 'ContentType': 'text/csv',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0x7f3ce23c4f90>}

In [18]:
dataset_bytes = dataset_object["Body"].read()
bytes_stream = io.BytesIO(dataset_bytes)

# Read from byte stream to create a dataframe
counties_df = pd.read_csv(bytes_stream, header=0, delimiter=',') 

In [19]:
# Exploring dataframe rows
counties_df.head(5)

,CensusId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001,Alabama,Autauga,55221,26745,28476,2.6,75.8,18.5,0.4,...,0.5,1.3,1.8,26.5,23986,73.6,20.9,5.5,0.0,7.6
1,1003,Alabama,Baldwin,195121,95314,99807,4.5,83.1,9.5,0.6,...,1.0,1.4,3.9,26.4,85953,81.5,12.3,5.8,0.4,7.5
2,1005,Alabama,Barbour,26932,14497,12435,4.6,46.2,46.7,0.2,...,1.8,1.5,1.6,24.1,8597,71.8,20.8,7.3,0.1,17.6
3,1007,Alabama,Bibb,22604,12073,10531,2.2,74.5,21.4,0.4,...,0.6,1.5,0.7,28.8,8294,76.8,16.1,6.7,0.4,8.3
4,1009,Alabama,Blount,57710,28512,29198,8.6,87.9,1.5,0.3,...,0.9,0.4,2.3,34.9,22189,82.0,13.5,4.2,0.4,7.7


In [30]:
# Statistics
print(f'There are {len(counties_df.columns)} columns: ' + ', '.join(counties_df.columns) + '.')

counties_df.describe()

There are 37 columns: CensusId, State, County, TotalPop, Men, Women, Hispanic, White, Black, Native, Asian, Pacific, Citizen, Income, IncomeErr, IncomePerCap, IncomePerCapErr, Poverty, ChildPoverty, Professional, Service, Office, Construction, Production, Drive, Carpool, Transit, Walk, OtherTransp, WorkAtHome, MeanCommute, Employed, PrivateWork, PublicWork, SelfEmployed, FamilyWork, Unemployment.


,CensusId,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
count,3220.000000,3.220000e+03,3.220000e+03,3.220000e+03,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000,...,3220.000000,3220.000000,3220.000000,3220.000000,3.220000e+03,3220.000000,3220.000000,3220.000000,3220.000000,3220.000000
mean,31393.605280,9.940935e+04,4.889694e+04,5.051241e+04,11.011522,75.428789,8.665497,1.723509,1.229068,0.082733,...,3.323509,1.612733,4.631770,23.278758,4.559352e+04,74.219348,17.560870,7.931801,0.288106,8.094441
std,16292.078954,3.193055e+05,1.566813e+05,1.626620e+05,19.241380,22.932890,14.279122,7.253115,2.633079,0.734931,...,3.756096,1.670988,3.178772,5.600466,1.496995e+05,7.863188,6.510354,3.914974,0.455137,4.096114
min,1001.000000,8.500000e+01,4.200000e+01,4.300000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,4.900000,6.200000e+01,25.000000,5.800000,0.000000,0.000000,0.000000
25%,19032.500000,1.121800e+04,5.637250e+03,5.572000e+03,1.900000,64.100000,0.500000,0.100000,0.200000,0.000000,...,1.400000,0.900000,2.700000,19.500000,4.550750e+03,70.500000,13.100000,5.400000,0.100000,5.500000
50%,30024.000000,2.603500e+04,1.293200e+04,1.305700e+04,3.900000,84.100000,1.900000,0.300000,0.500000,0.000000,...,2.400000,1.300000,3.900000,23.000000,1.050800e+04,75.700000,16.200000,6.900000,0.200000,7.600000
75%,46105.500000,6.643050e+04,3.299275e+04,3.348750e+04,9.825000,93.200000,9.600000,0.600000,1.200000,0.000000,...,4.000000,1.900000,5.600000,26.800000,2.863275e+04,79.700000,20.500000,9.400000,0.300000,9.900000
max,72153.000000,1.003839e+07,4.945351e+06,5.093037e+06,99.900000,99.800000,85.900000,92.100000,41.600000,35.300000,...,71.200000,39.100000,37.200000,44.000000,4.635465e+06,88.300000,66.200000,36.600000,9.800000,36.500000


## Exploratory Data Analysis (EDA)

Some of the columns represent numbers in absolute value, while others are shown in percentage. Some administrative data (such as `CensusId`) do not bring any important information to the model. Also, the dataset index must be defined to represent a meaningful and unique information, so the `State-County` pair will be used.

### Pre-processing data
#### Objectives:
- Drop incomplete rows
- Re-index dataframe to `State-County`
- Drop useless / non-quantitative data (`State`, `County`, `CensusId`)

In [34]:
clean_counties_df = counties_df.dropna()
print(f'Total rows before dropna: {len(counties_df)}, after dropna: {len(clean_counties_df)}')

print(f'Dropped rows: {len(counties_df)-len(clean_counties_df)}')

Total rows before dropna: 3220, after dropna: 3218
Dropped rows: 2


In [ ]:
clean_counties_df.index = clean_counties_df['State'] + '-' + clean_counties_df['County']

In [ ]:
clean_counties_df = clean_counties_df.drop(columns=['State', 'County', 'CensusId'])

Remaining features are:

In [1]:
# features
features_list = clean_counties_df.columns.values
print('Features: \n', features_list)
clean_counties_df.head(5)

NameError: name 'clean_counties_df' is not defined

## Data Visualization

After data pre-processing, I want to visualize and explore the dataset further. For this, I plot histograms depicting the distributions of the datapoints. 

**The frequencies (y-axis) in the plots represent the number of counties that fall into each range of values (bins) for a specific quantitative column**.

In [ ]:
# Definition of histograms to be generated
columns_to_hist = ['Men', 'Women', 'Hispanic', 'White', 'Black', 'Native', 'Asian']
n_bins = 30


for column_name in columns_to_hist:
    ax=plt.subplots(figsize=(6,3))
    # get data by column_name and display a histogram
    ax = plt.hist(clean_counties_df[column_name], bins=n_bins)
    title=f"Histogram of {column_name}"
    plt.title(title, fontsize=12)
    plt.show()


## Data normalization (Pre-training)

Since the dataset columns present distinct range of values, at different scales, I use an MinMaxScaler to standardize numerical values.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# scale numerical features into a normalized range, 0-1
# store them in this dataframe
counties_scaled = clean_counties_df.copy()
for column in counties_scaled:
    counties_scaled[[column]] = MinMaxScaler().fit_transform(clean_counties_df[[column]])
    
counties_scaled.describe()

## Get session, IAM role and bucket

In [ ]:
session = sagemaker.Session() # SageMaker session

role = sagemaker.get_execution_role() # IAM role for this session

bucket_name = session.default_bucket() # Bucket

## Principal Component Analysis

In [ ]:
# define a folder name for this job's results
prefix = 'counties'
# Output path in S3
output_path=f's3://{bucket_name}/{prefix}/'

In [ ]:
# define a PCA model
from sagemaker import PCA

N_COMPONENTS=33

pca_estimator = PCA(role=role,
             train_instance_count=1,
             train_instance_type='ml.c4.xlarge',
             output_path=output_path,
             num_components=N_COMPONENTS, 
             sagemaker_session=session)

In [ ]:
# Prepare RecordSet (required by SageMaker PCA Model)

train_data_np = counties_scaled.values.astype('float32')

formatted_train_data = pca_estimator.record_set(train_data_np)

In [ ]:
# Train model

%%time

pca_estimator.fit(formatted_train_data)

In [ ]:
# Retrieve job results from S3
pca_job_name = 'pca-2020-09-25-20-39-46-297'

path_to_model = os.path.join(prefix, training_job_name, 'output/model.tar.gz')

# download and unzip model
boto3.resource('s3').Bucket(bucket_name).download_file(model_key, 'model.tar.gz')
# unzipping as model_algo-1
os.system('tar -zxvf model.tar.gz')
os.system('unzip model_algo-1')

In [ ]:
# Load MXNet model artifacts into a local MXNet
import mxnet as mx

pca_params = mx.ndarray.load('model_algo-1')

### PCA Attributes

Three types of model attributes are contained within the PCA model.

- `mean`: The mean that was subtracted from a component in order to center it.
- `v`: The makeup of the principal components; (same as ‘components_’ in an sklearn PCA model).
- `s`: The singular values of the components for the PCA transformation. This does not exactly give the % variance from the original feature space, but can give the % variance from the projected feature space.

We are only interested in `v` and `s`.

In [ ]:
# Get the necessary attributes

pca_v = pd.DataFrame(pca_params['v'].asnumpy())
pca_s = pd.DataFrame(pca_params['s'].asnumpy())

In [ ]:
# The components are ordered in ascending order of s
print(pca_s)

### Dimensionality and Variance
I want to keep the `n` most important features so that they totalize at least 80% of the original variance. Hopefully, a much smaller number of features will be necessary to cover most of the data variance, and those are the components to be picked in order to reduce the model dimensionality.

The variance of a set of `n` components is given as:

\begin{equation}
\frac{\sum_{i}^{n} {s_i}^2}{\sum s^2}
\end{equation}

In [ ]:
# Calculate the explained variance for the top n principal components
def explained_variance(s, n_top_components):
    '''Calculates the approx. data variance that n_top_components captures.
       :param s: A dataframe of singular values for top components; 
           the top value is in the last row.
       :param n_top_components: An integer, the number of top components to use.
       :return: The expected data variance covered by the n_top_components.'''
    
    s2 = s * s
    last_n_variance = s2.iloc[-n_top_components:].sum()[0]
    total_variance = s2.iloc[:].sum()[0]
    
    return last_n_variance/total_variance

In [ ]:
### Testing 
n_top_components = 7 # select a value for the number of top components

# calculate the explained variance
exp_variance = explained_variance(s, n_top_components)
print('Explained variance: ', exp_variance)

### Composition of PCA components


In [ ]:
import seaborn as sns

def display_component(v, features_list, component_num, n_weights=10):
    
    # get index of component (last row - component_num)
    row_idx = N_COMPONENTS-component_num

    # get the list of weights from a row in v, dataframe
    v_1_row = v.iloc[:, row_idx]
    v_1 = np.squeeze(v_1_row.values)

    # match weights to features in counties_scaled dataframe, using list comporehension
    comps = pd.DataFrame(list(zip(v_1, features_list)), 
                         columns=['weights', 'features'])

    # we'll want to sort by the largest n_weights
    # weights can be neg/pos and we'll sort by magnitude
    comps['abs_weights']=comps['weights'].apply(lambda x: np.abs(x))
    sorted_weight_data = comps.sort_values('abs_weights', ascending=False).head(n_weights)

    # display using seaborn
    ax=plt.subplots(figsize=(10,6))
    ax=sns.barplot(data=sorted_weight_data, 
                   x="weights", 
                   y="features", 
                   palette="Blues_d")
    ax.set_title("PCA Component Makeup, Component #" + str(component_num))
    plt.show()


In [ ]:
# Show composition of a given component
num=1
display_component(pca_v, counties_scaled.columns.values, component_num=num, n_weights=15)

## Deploying PCA to evaluate dataset



In [ ]:
%%time
# this takes a little while, around 7mins
pca_predictor = pca_SM.deploy(initial_instance_count=1,instance_type='ml.t2.medium')

We can pass the original, numpy dataset to the model and transform the data using the model we created. Then we can take the largest n components to reduce the dimensionality of our data.

In [ ]:
# pass np train data to the PCA model
train_pca = pca_predictor.predict(train_data_np)

In [ ]:
# create dimensionality-reduced data
def create_transformed_df(train_pca, counties_scaled, n_top_components):
    ''' Return a dataframe of data points with component features. 
        The dataframe should be indexed by State-County and contain component values.
        :param train_pca: A list of pca training data, returned by a PCA model.
        :param counties_scaled: A dataframe of normalized, original features.
        :param n_top_components: An integer, the number of top components to use.
        :return: A dataframe, indexed by State-County, with n_top_component values as columns.        
     '''
    # create a dataframe of component features, indexed by State-County
    new_df = pd.DataFrame()
    
    # your code here
    for row in range(len(train_pca)):
        for col in range(n_top_components):
            new_df.loc[counties_scaled.index[row], f'c_{col+1}'] = train_pca[row].label['projection'].float32_tensor.values[-col-1]
    
    return new_df

In [ ]:
## Specify top n
top_n = n_top_components

# call your function and create a new dataframe
counties_transformed = create_transformed_df(train_pca, counties_scaled, n_top_components=top_n)


# print result
counties_transformed.head()

### Delete PCA endpoint


In [ ]:
# delete predictor endpoint
session.delete_endpoint(pca_predictor.endpoint)

---
## Population Segmentation

The transformed model now should be used to feed our unsupervised learning model. 
For the clusterization, I'll apply the `K-means` clusterization.

In [ ]:
# define a KMeans estimator
from sagemaker import KMeans
estimator = KMeans(
    role, 
    train_instance_count=1, 
    train_instance_type='ml.c4.xlarge', 
    k=8,
    sagemaker_session=session, 
    output_path=f's3://{bucket_name}/{prefix}/')

In [ ]:
# Create RecordSet

train_data_np = counties_transformed.values.astype('float32')

# convert to RecordSet format
train_data_rs = estimator.record_set(train_data_np)

In [ ]:
# Train model
%%time
estimator.fit(records=train_data_rs, wait=True)

In [ ]:
# Deploy model
%%time
kmeans_predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

In [ ]:
# get the predicted clusters for all the kmeans training data
cluster_info = kmeans_predictor.predict(train_data_np)

## Exploring clusters

In [ ]:
# print cluster info for a given data point
data_idx = 0

print('County is: ', counties_transformed.index[data_idx])
print()
print(cluster_info[data_idx])

### Distribution of data over clusters


In [ ]:
# get all cluster labels
cluster_labels = [c.label['closest_cluster'].float32_tensor.values[0] for c in cluster_info]

# count up the points in each cluster
cluster_df = pd.DataFrame(cluster_labels)[0].value_counts()

print(cluster_df)

In [ ]:
# Delete endpoint
session.delete_endpoint(kmeans_predictor.endpoint)

---
# Model Attributes and Explainability

Explaining the result of the modeling is an important step in making use of our analysis. By combining PCA and k-means, and the information contained in the model attributes within a SageMaker trained model, we can learn about a population and remark on some patterns we've found, based on the data.

In [ ]:
# download and unzip the kmeans model file
training_job_name='kmeans-2020-09-30-15-26-50-572'

# where the model is saved, by default
model_key = os.path.join(prefix, training_job_name, 'output/model.tar.gz')
print(model_key)

# download and unzip model
boto3.resource('s3').Bucket(bucket_name).download_file(model_key, 'model.tar.gz')

# unzipping as model_algo-1
os.system('tar -zxvf model.tar.gz')
os.system('unzip model_algo-1')

In [ ]:
# get the trained kmeans params using mxnet
kmeans_model_params = mx.ndarray.load('model_algo-1')

# what are the params
print(kmeans_model_params)

There is only 1 set of model parameters contained within the k-means model: the cluster centroid locations in PCA-transformed, component space.

- **centroids**: The location of the centers of each cluster in component space, identified by the k-means algorithm.

In [ ]:
# get all the centroids
cluster_centroids=pd.DataFrame(kmeans_model_params[0].asnumpy())
cluster_centroids.columns=counties_transformed.columns

display(cluster_centroids)

### Visualization of Centroids

In [ ]:
# generate a heatmap in component space, using the seaborn library
plt.figure(figsize = (12,9))
ax = sns.heatmap(cluster_centroids.T, cmap = 'YlGnBu')
ax.set_xlabel("Cluster")
plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)
ax.set_title("Attribute Value by Centroid")
plt.show()

### Natural Groupings

In [ ]:
# add a 'labels' column to the dataframe
counties_transformed['labels']=list(map(int, cluster_labels))

# sort by cluster label 0-6
sorted_counties = counties_transformed.sort_values('labels', ascending=True)
# view some pts in cluster 0
sorted_counties.head(20)

---


## Delete buckets  

In [5]:
!aws s3 rb $bucket_name --force

usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help
aws: error: the following arguments are required: path
